In [ ]:
import xml.etree.ElementTree as ET
import pickle
import os
import cv2
from os import getcwd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imgaug as ia
from imgaug import augmenters as iaa

# 影像增量(optinal)

In [ ]:
IMG_DIR = './dataset/data_mango_select/train'
XML_DIR = './dataset/data_mango_select/train_anno'

AUG_XML_DIR = "./dataset/data_mango_select/train_aug_anno"  # 存储增强后的XML文件夹路径
mkdir(AUG_XML_DIR)

AUG_IMG_DIR = "./dataset/data_mango_select/train_aug"  # 存储增强后的影像文件夹路径
mkdir(AUG_IMG_DIR)

AUGLOOP = 5 # 每张影像增强的数量

In [ ]:


ia.seed(1)

def read_xml_annotation(root, image_id):
    in_file = open(os.path.join(root, image_id))
    tree = ET.parse(in_file)
    root = tree.getroot()
    bndboxlist = []

    for object in root.findall('object'):  # 找到root节点下的所有country节点
        bndbox = object.find('bndbox')  # 子节点下节点rank的值

        xmin = int(bndbox.find('xmin').text)
        xmax = int(bndbox.find('xmax').text)
        ymin = int(bndbox.find('ymin').text)
        ymax = int(bndbox.find('ymax').text)
        # print(xmin,ymin,xmax,ymax)
        bndboxlist.append([xmin,ymin,xmax,ymax])
        # print(bndboxlist)

    bndbox = root.find('object').find('bndbox')
    return bndboxlist
# (506.0000, 330.0000, 528.0000, 348.0000) -> (520.4747, 381.5080, 540.5596, 398.6603)
def change_xml_annotation(root, image_id, new_target):
    new_xmin = new_target[0]
    new_ymin = new_target[1]
    new_xmax = new_target[2]
    new_ymax = new_target[3]

    in_file = open(os.path.join(root, str(image_id) + '.xml'))  # 这里root分别由两个意思
    tree = ET.parse(in_file)
    xmlroot = tree.getroot()
    object = xmlroot.find('object')
    bndbox = object.find('bndbox')
    xmin = bndbox.find('xmin')
    xmin.text = str(new_xmin)
    ymin = bndbox.find('ymin')
    ymin.text = str(new_ymin)
    xmax = bndbox.find('xmax')
    xmax.text = str(new_xmax)
    ymax = bndbox.find('ymax')
    ymax.text = str(new_ymax)
    tree.write(os.path.join(root, str(image_id) + "_aug" + '.xml'))

def change_xml_list_annotation(root, image_id, new_target,saveroot,id):

    in_file = open(os.path.join(root, str(image_id) + '.xml'))  # 这里root分别由两个意思
    tree = ET.parse(in_file)
    xmlroot = tree.getroot()
    index = 0

    for object in xmlroot.findall('object'):  # 找到root节点下的所有country节点
        bndbox = object.find('bndbox')  # 子节点下节点rank的值

        # xmin = int(bndbox.find('xmin').text)
        # xmax = int(bndbox.find('xmax').text)
        # ymin = int(bndbox.find('ymin').text)
        # ymax = int(bndbox.find('ymax').text)

        new_xmin = new_target[index][0]
        new_ymin = new_target[index][1]
        new_xmax = new_target[index][2]
        new_ymax = new_target[index][3]

        xmin = bndbox.find('xmin')
        xmin.text = str(new_xmin)
        ymin = bndbox.find('ymin')
        ymin.text = str(new_ymin)
        xmax = bndbox.find('xmax')
        xmax.text = str(new_xmax)
        ymax = bndbox.find('ymax')
        ymax.text = str(new_ymax)

        index = index + 1

    tree.write(os.path.join(saveroot, str(image_id) + "_aug_" + str(id) + '.xml'))


def mkdir(path):

    # 去除首位空格
    path = path.strip()
    # 去除尾部 \ 符号
    path = path.rstrip("\\")
    # 判断路径是否存在
    # 存在     True
    # 不存在   False
    isExists = os.path.exists(path)
    # 判断结果
    if not isExists:
        # 如果不存在则创建目录
         # 创建目录操作函数
        os.makedirs(path)
        print(path + ' 创建成功')
        return True
    else:
        # 如果目录存在则不创建，并提示目录已存在
        print(path + ' 目录已存在')
        return False



In [ ]:
if __name__ == "__main__":

    boxes_img_aug_list = []
    new_bndbox = []
    new_bndbox_list = []


    # 影像增强
    seq = iaa.Sequential([
        iaa.Flipud(0.5),  # vertically flip 20% of all images
        iaa.Fliplr(0.5), # 水平翻轉影象
#         iaa.Fliplr(0.5),  # 镜像
#         iaa.GaussianBlur(sigma=(0, 0.5)), # iaa.GaussianBlur(0.5),
#         iaa.Rot90(1),
        iaa.Affine(rotate=(-20, 20)),
        iaa.Affine(
#             translate_px={"x": 15, "y": 15},
            scale=(1.5, 2),
#             rotate=(-30, 30)
        )  # translate by 40/60px on x/y axis, and scale to 50-70%, affects BBs
    ])

    for root, sub_folders, files in os.walk(XML_DIR):

        for name in files:

            bndbox = read_xml_annotation(XML_DIR, name)

            for epoch in range(AUGLOOP):
                seq_det = seq.to_deterministic()  # 保持坐标和图像同步改变，而不是随机

                # 读取图片
                img = Image.open(os.path.join(IMG_DIR, name[:-4] + '.jpg'))
                img = np.array(img)
                print(img.shape)
                print(img.shape[0])
                print(img.shape[1])
                # bndbox 坐标增强
                for i in range(len(bndbox)):
                    bbs = ia.BoundingBoxesOnImage([
                        ia.BoundingBox(x1=bndbox[i][0], y1=bndbox[i][1], x2=bndbox[i][2], y2=bndbox[i][3]),
                    ], shape=img.shape)

                    bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
                    boxes_img_aug_list.append(bbs_aug)
                    if(int(bbs_aug.bounding_boxes[0].x1)<0):
                        x1 = 0
                    elif(int(bbs_aug.bounding_boxes[0].x1)>img.shape[1]):
                        x1 = img.shape[1]
                    else:
                        x1 = int(bbs_aug.bounding_boxes[0].x1)
                    
                    if(int(bbs_aug.bounding_boxes[0].y1)<0):
                        y1 = 0
                    elif(int(bbs_aug.bounding_boxes[0].y1)>img.shape[0]):
                        y1 = img.shape[0]
                    else:
                        y1 = int(bbs_aug.bounding_boxes[0].y1)
                    
                    if(int(bbs_aug.bounding_boxes[0].x2)>img.shape[1]):
                        x2 = img.shape[1]
                    elif(int(bbs_aug.bounding_boxes[0].x2)<0):
                        x2 = 0
                    else:
                        x2 = int(bbs_aug.bounding_boxes[0].x2)
                    
                    if(int(bbs_aug.bounding_boxes[0].y2)>img.shape[0]):
                        y2 = img.shape[0]
                    elif(int(bbs_aug.bounding_boxes[0].y2)<0):
                        y2 = 0
                    else :
                        y2 = int(bbs_aug.bounding_boxes[0].y2)
                    
                    # new_bndbox_list:[[x1,y1,x2,y2],...[],[]]
                    new_bndbox_list.append([x1,y1,x2,y2])
                # 存储变化后的图片
                image_aug = seq_det.augment_images([img])[0]
                path = os.path.join(AUG_IMG_DIR, str(name[:-4]) + "_aug_" + str(epoch+10) + '.jpg')
                # image_auged = bbs.draw_on_image(image_aug, thickness=0)
                Image.fromarray(image_aug).save(path)

                # 存储变化后的XML
                change_xml_list_annotation(XML_DIR, name[:-4], new_bndbox_list,AUG_XML_DIR,epoch+10)
                print(str(name[:-4]) + "_aug_" + str(epoch+5) + '.jpg')
                new_bndbox_list = []

# 查看增量後的影像、bbnx(optinal)

In [ ]:
from xml.dom import minidom
import xml.etree.ElementTree as ET

import time
from IPython.display import clear_output

for img in os.listdir(AUG_IMG_DIR):
    print(img)
    f_name = img.split('.')[0]
    tree = ET.parse(os.path.join(AUG_XML_DIR,f_name+".xml").replace('\\','/'))
    root = tree.getroot()
    
#     labelXML = minidom.parse()
    labelXmin = []
    labelYmin = []
    labelXmax = []
    labelYmax = []
    img = cv2.imread(os.path.join(AUG_IMG_DIR,img).replace('\\','/'),1)
#     plt.imshow(img)
    print(img.shape)
    for elem in root.findall('object'):
        for bndbox in elem.findall('bndbox'):
            x = bndbox.find('xmin').text
            y = bndbox.find('ymin').text
            xmax = bndbox.find('xmax').text
            ymax = bndbox.find('ymax').text
            print(x,y,xmax,ymax)
            cv2.rectangle(img,(int(x),int(y)),(int(xmax),int(ymax)),(0,0,255),2)
    plt.imshow(img)
    plt.show()
    time.sleep(5)
    clear_output()

# 1_labels_to_yolo_format

In [ ]:
print(AUG_XML_DIR)
print(AUG_IMG_DIR)

In [ ]:
import glob, os
import os.path
import time
from shutil import copyfile
import cv2
from xml.dom import minidom
from os.path import basename
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

xmlFolder = AUG_XML_DIR #xml_DIR
imgFolder = AUG_IMG_DIR #img DIR
#negFolder = "/home/chtseng/datasets/12_hand_gestures/negatives"
negFolder = ""
saveYoloPath = "./dataset/data_mango_select/train_aug_txt" #yolo_dataset_save_path

classList = { #classes name
    'A':0,
    'B':1,
    'C':2,
}

#---------------------------------------------------------------------

if not os.path.exists(saveYoloPath):
    os.makedirs(saveYoloPath)

def transferYolo( xmlFilepath, imgFilepath):
    global imgFolder

    img_file, img_file_extension = os.path.splitext(imgFilepath)
    img_filename = basename(img_file)
    print(imgFilepath)

    if(xmlFilepath is not None):
        img = cv2.imread(imgFilepath)
        imgShape = img.shape
        print (img.shape)
        img_h = imgShape[0]
        img_w = imgShape[1]

        labelXML = minidom.parse(xmlFilepath)
        labelName = []
        labelXmin = []
        labelYmin = []
        labelXmax = []
        labelYmax = []
        totalW = 0
        totalH = 0
        countLabels = 0

        tmpArrays = labelXML.getElementsByTagName("filename")
        for elem in tmpArrays:
            filenameImage = elem.firstChild.data

        tmpArrays = labelXML.getElementsByTagName("name")
        for elem in tmpArrays:
            labelName.append(str(elem.firstChild.data))

        tmpArrays = labelXML.getElementsByTagName("xmin")
        for elem in tmpArrays:
            labelXmin.append(int(elem.firstChild.data))

        tmpArrays = labelXML.getElementsByTagName("ymin")
        for elem in tmpArrays:
            labelYmin.append(int(elem.firstChild.data))

        tmpArrays = labelXML.getElementsByTagName("xmax")
        for elem in tmpArrays:
            labelXmax.append(int(elem.firstChild.data))

        tmpArrays = labelXML.getElementsByTagName("ymax")
        for elem in tmpArrays:
            labelYmax.append(int(elem.firstChild.data))

        yoloFilename = os.path.join(saveYoloPath, img_filename + ".txt").replace('\\','/')
        print("writeing to {}".format(yoloFilename))

        with open(yoloFilename, 'a') as the_file:
            i = 0
            for className in labelName:
                if(className in classList):
                    classID = classList[className]
                    x = (labelXmin[i] + (labelXmax[i]-labelXmin[i])/2) * 1.0 / img_w 
                    y = (labelYmin[i] + (labelYmax[i]-labelYmin[i])/2) * 1.0 / img_h
                    w = (labelXmax[i]-labelXmin[i]) * 1.0 / img_w
                    h = (labelYmax[i]-labelYmin[i]) * 1.0 / img_h
                    if(w>1):
                        w = 1
                    if(h>1):
                        h = 1
                    if(w<0):
                        w = 0
                    if(h<0):
                        h = 0
                    # print(x,y,w,h)

                    the_file.write(str(classID) + ' ' + str(x) + ' ' + str(y) + ' ' + str(w) + ' ' + str(h) + '\n')
                    i += 1

    else:
        yoloFilename = os.path.join(saveYoloPath ,img_filename + ".txt").replace('\\','/')
        print("writeing negative file to {}".format(yoloFilename))

        with open(yoloFilename, 'a') as the_file:
            the_file.write('')

    the_file.close()

#---------------------------------------------------------------
fileCount = 0

for file in os.listdir(imgFolder):
    filename, file_extension = os.path.splitext(file)
    file_extension = file_extension.lower()

    if(file_extension == ".jpg" or file_extension==".png" or file_extension==".jpeg" or file_extension==".bmp"):
        imgfile = os.path.join(imgFolder, file).replace('\\','/')
        xmlfile = os.path.join(xmlFolder ,filename + ".xml").replace('\\','/')

        if(os.path.isfile(xmlfile)):
            print("id:{}".format(fileCount))
            print("processing {}".format(imgfile))
            print("processing {}".format(xmlfile))
            fileCount += 1

            transferYolo( xmlfile, imgfile)
            copyfile(imgfile, os.path.join(saveYoloPath ,file).replace('\\','/'))

if(os.path.exists(negFolder)):
    for file in os.listdir(negFolder):
        filename, file_extension = os.path.splitext(file)
        file_extension = file_extension.lower()
        imgfile = os.path.join(negFolder ,file).replace('\\','/')

        if(file_extension == ".jpg" or file_extension==".png" or file_extension==".jpeg" or file_extension==".bmp"):
            transferYolo( None, imgfile, "")
            copyfile(imgfile, os.path.join(saveYoloPath ,file).replace('\\','/'))


# 2_split_train_test

In [ ]:
import random
import glob, os
import os.path

#---------------------------------------------------------
testRatio = 0.1 #切割比例
train_dir = './dataset/train_txt/'
test_dir = './dataset/test_txt/'
cfgFolder = "./yolov5_config/"
train_out_name = "train_data.txt"
val_out_name = "val_data.txt"
test_out_name = "test_data.txt"
#--------------------------------------------------------

def make_train_test_val(imageFolder,cfgFolder,train_out_name,test_out_name,testRatio):
    fileList = []
    outputTrainFile = os.path.join(cfgFolder,train_out_name).replace('\\','/')
    outputTestFile = os.path.join(cfgFolder ,test_out_name).replace('\\','/')

    if not os.path.exists(cfgFolder):
        os.makedirs(cfgFolder)

    for file in os.listdir(imageFolder):
        filename, file_extension = os.path.splitext(file)
        file_extension = file_extension.lower()

        if(file_extension == ".jpg" or file_extension==".jpeg" or file_extension==".png" or file_extension==".bmp"):
            fileList.append(os.path.join(imageFolder, file).replace('\\','/'))

    print("total image files: ", len(fileList))

    testCount = int(len(fileList) * testRatio)
    trainCount = len(fileList) - testCount

    a = range(len(fileList))
    test_data = random.sample(a, testCount)
    #train_data = random.sample(a, trainCount)
    train_data = [x for x in a if x not in test_data]

    print ("Train:{} images".format(len(train_data)))
    print("Test:{} images".format(len(test_data)))

    with open(outputTrainFile, 'a') as the_file:
        for i in train_data:
            the_file.write(fileList[i] + "\n")

    the_file.close()
    if(testRati!=0):
        with open(outputTestFile, 'a') as the_file:
            for i in test_data:
                the_file.write(fileList[i] + "\n")

        the_file.close()
make_train_test_val(train_dir,cfgFolder,train_out_name,val_out_name,testRatio)
make_train_test_val(test_dir,cfgFolder,test_out_name,"",0)


# 製作anchors

In [ ]:
# -*- coding=utf-8 -*-

from utils.general import *

yaml_path = './dataset/mango_select_aug.yaml'
n=9
img_size=640
thr=4.0
gen=1000
verbose=True

print(kmean_anchors(yaml_path,n,img_size,thr,gen,verbose))
